In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.stats import pearsonr
from sklearn.frozen import FrozenEstimator
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
### Update with joined geno-pheno data

chunksize = 100

list_of_dataframes = []

for df in pd.read_csv('DarpaQCFiltered.csv', chunksize=chunksize, index_col=0):
    list_of_dataframes.append(df)

result = pd.concat(list_of_dataframes)
df = result
df

# ax_columns = [col for col in df.columns if col.startswith('AX')]
# # len(ax_columns)
# X = df[ax_columns]
# y = df["Status"]
# X = X.to_numpy()
# y = y.to_numpy()

# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X

AX-564298109  AX-574114011  AX-577073921  AX-564298209  \
AX-574114010                                                           
0.0                    0.0           0.0           0.0           0.0   
0.0                    0.0           0.0           2.0           2.0   
0.0                    0.0           1.0           0.0           1.0   
0.0                    0.0           2.0           2.0           1.0   
0.0                    0.0           2.0           2.0           2.0   
...                    ...           ...           ...           ...   
0.0                    0.0           1.0           1.0           0.0   
1.0                    1.0           0.0           0.0           0.0   
0.0                    0.0           1.0           2.0           0.0   
0.0                    0.0           0.0           0.0           0.0   
2.0                    2.0           0.0           0.0           0.0   

              AX-574114024  AX-576891682  AX-564298228  AX-576891683  \
AX-574114010                                                           
0.0                    0.0           0.0          1.58          1.61   
0.0                    2.0           2.0          2.00          2.00   
0.0                    1.0           1.0          1.00          1.00   
0.0                    1.0           1.0          1.00          1.00   
0.0                    2.0           2.0          2.00          2.00   
...                    ...           ...           ...           ...   
0.0                    0.0           1.0          1.00          1.00   
1.0                    0.0           0.0          2.00          2.00   
0.0                    1.0           1.0          1.00          1.00   
0.0                    0.0           0.0          2.00          2.00   
2.0                    0.0           0.0          2.00          2.00   

              AX-574396910  AX-574130565  ...  AX-568137573  AX-576870369  \
AX-574114010                              ...                               
0.0                    1.0           0.0  ...           0.0           0.0   
0.0                    0.0           0.0  ...           0.0           2.0   
0.0                    1.0           0.0  ...           2.0           1.0   
0.0                    0.0           0.0  ...           0.0           2.0   
0.0                    1.0           0.0  ...           1.0           1.0   
...                    ...           ...  ...           ...           ...   
0.0                    1.0           2.0  ...           0.0           1.0   
1.0                    2.0           0.0  ...           0.0           2.0   
0.0                    1.0           0.0  ...           1.0           2.0   
0.0                    2.0           1.0  ...           0.0           2.0   
2.0                    2.0           1.0  ...           0.0           2.0   

              AX-568137858  AX-574092827  AX-568138198  AX-574092919  \
AX-574114010                                                           
0.0                    0.0          1.62           0.0           0.0   
0.0                    0.0          1.00           1.0           1.0   
0.0                    0.0          2.00           0.0           0.0   
0.0                    0.0          1.00           0.0           1.0   
0.0                    1.0          1.00           0.0           0.0   
...                    ...           ...           ...           ...   
0.0                    0.0          1.00           0.0           0.0   
1.0                    0.0          2.00           0.0           0.0   
0.0                    0.0          1.00           1.0           1.0   
0.0                    0.0          2.00           0.0           0.0   
2.0                    0.0          1.00           0.0           0.0   

              AX-568138441  AX-574093181  AX-574093191  AX-574093227  
AX-574114010                                                          
0.0                   1.00           0.0           1.0     